# Exact Method
Using Gurobi solver

In [2]:
import math
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [35]:
df = pd.read_csv('datasets/compas_metadata.csv').drop(columns = ['instances'])
display(df)
performance_columns = [col for col in df.columns if col.startswith('algo')]
X = df.drop(columns = performance_columns)
Y = df[performance_columns][['algo_bagging']]

F = X.to_numpy().transpose()
Y = Y.to_numpy()

n = X.shape[1] # number of instances
m = Y.shape[0] # number of features
print(m, n)

print(Y.shape)

,feature_CL,feature_CLD,feature_DCP,feature_F1,feature_Harmfulness,feature_N1,feature_N2,feature_TD_P,feature_TD_U,feature_kDN,algo_bagging,algo_gradient_boosting,algo_logistic_regression,algo_mlp,algo_random_forest,algo_svc_linear,algo_svc_rbf
0,0.817262,0.817262,0.645463,0.076923,0.002014,1.0,0.000000,0.75,0.346154,0.6,0.783429,1.294225,1.086574,1.138194,1.152911,1.098590,1.218375
1,0.061321,0.061321,0.310160,0.230769,0.000403,0.0,1.000000,0.75,0.576923,0.2,1.038007,0.228349,0.327910,0.300339,0.204690,0.313141,0.219436
2,0.144556,0.144556,0.348606,0.153846,0.000000,0.0,0.408284,0.75,0.692308,0.2,0.437654,0.395576,0.268317,0.245394,0.490364,0.268453,0.328142
3,0.004853,0.004853,0.234506,0.076923,0.000000,0.0,0.000000,0.75,0.500000,0.2,0.316959,0.248446,0.233864,0.229492,0.219892,0.227627,0.327807
4,0.075165,0.075165,0.354537,0.076923,0.000000,1.0,0.000000,0.75,0.500000,0.2,0.648114,0.352047,0.527617,0.512862,0.428196,0.522380,0.371585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5273,0.927173,0.927173,0.645463,0.076923,0.001611,1.0,0.000000,0.75,0.461538,0.6,0.978772,1.316957,1.040314,1.087224,1.245815,1.043946,1.207007
5274,0.406834,0.406834,0.518750,0.000000,0.000000,1.0,0.000000,1.00,0.423077,0.8,0.703149,0.871755,0.787233,0.819198,0.774348,0.827964,0.816926
5275,0.404776,0.404776,0.281915,0.076923,0.000000,0.0,0.000000,1.00,0.384615,0.4,0.505306,0.452444,0.732361,0.748449,0.632225,0.763739,0.997945
5276,0.404776,0.404776,0.281915,0.076923,0.000716,0.0,0.000000,1.00,0.384615,0.4,0.475926,0.446878,0.710063,0.724876,0.627801,0.757990,0.459586


5278 10
(5278, 1)


In [50]:

try:

    # Create a new model
    mod = gp.Model("matrix1") 

    # Create variables
    Ar = mod.addMVar(shape=(2, m), vtype=GRB.CONTINUOUS, name="Ar") 
    Br = mod.addMVar(shape=(m, 2), vtype=GRB.CONTINUOUS, name="Br")
    cr = mod.addMVar(shape= (2, 1), vtype=GRB.CONTINUOUS, name="cr")
    
    # Set objective
    mod.setObjective(np.sum(np.square(F - np.matmul(Br, np.matmul(Ar, F)))) + 
                   np.sum(np.square(Y.transpose - np.matmul(cr, np.matmul(Ar, F)))), GRB.MINIMIZE)


    # Add constraints
    
    
    # Optimize model
    mod.optimize()

    #print(x.X)
    print('Obj: %g' % mod.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')
    #print(e.message)

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)